<a href="https://colab.research.google.com/github/kolasyam/Fake-News-Prediction-using-Machine-Learning/blob/main/Fake_News_Prediction_using_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

About the Dataset:

1. id: unique id for a news article
2. title: the title of a news article
3. text: the text of the article; could be incomplete
4. label: a label that marks whether the news article is real or fake:
           1: Fake news
           0: real News

In [5]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [2]:
dfT = pd.read_csv('/content/True.csv.zip');
dfF = pd.read_csv('/content/Fake.csv.zip');

In [3]:
dfT["Label"]=1;
dfF["Label"]=0;

In [4]:
df = pd.concat([dfT,dfF],ignore_index=True)


In [6]:
df.head()

,title,text,subject,date,Label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017",1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017",1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017",1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017",1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017",1


In [7]:
df.isnull().sum()


,0
title,0
text,0
subject,0
date,0
Label,0


In [8]:
df.drop(["date"],axis=1)


,title,text,subject,Label
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,1
...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,0


In [30]:
x=df.iloc[:,:3]
y=df["Label"]
x.head()


,title,text,subject
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews


In [31]:
x["subject"].value_counts()

,count
subject,
politicsNews,11272
worldnews,10145
News,9050
politics,6841
left-news,4459
Government News,1570
US_News,783
Middle-east,778


In [33]:
dummy_subjects = pd.get_dummies(x["subject"],prefix="subject")
x=pd.concat([x,dummy_subjects],axis=1)
x.drop("subject",axis=1,inplace=True)

In [34]:
x.head()

,title,text,subject_Government News,subject_Middle-east,subject_News,subject_US_News,subject_left-news,subject_politics,subject_politicsNews,subject_worldnews
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,False,False,False,False,False,False,True,False
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,False,False,False,False,False,False,True,False
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,False,False,False,False,False,False,True,False
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,False,False,False,False,False,False,True,False
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,False,False,False,False,False,False,True,False


In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words='english', max_features=5000)  # optional: adjust max_features
X_text = tfidf.fit_transform(x['text'])

combined = x["title"]+" - "+x["text"]
X_text = tfidf.fit_transform(combined)

In [17]:
from scipy.sparse import csr_matrix, hstack

subject_sparse = csr_matrix(dummy_subjects.values)
X_final = hstack([X_text, subject_sparse])


In [36]:
X_final.toarray()


array([[0.        , 0.        , 0.05778322, ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.        , 0.01383297, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03593701, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.04132518, ..., 0.        , 0.        ,
        0.        ]])

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import cross_val_score

x_temp,x_test,y_temp,y_test = train_test_split(X_final,y,test_size=0.2,random_state=42)

In [38]:
x_train,x_val,y_train,y_val = train_test_split(x_temp,y_temp,test_size=0.2,random_state=42)


Logistic Regression

In [41]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Evaluation for Training Data

In [46]:
print("Evaluation for Logistic Regression")
x_pred=log_model.predict(x_train)
print(f"Accuracy is ",accuracy_score(y_train,x_pred))
print("Classification Report:\n", classification_report(y_train, x_pred))
print("Confusion Matrix:\n", confusion_matrix(y_train, x_pred))

Evaluation for Logistic Regression
Accuracy is  1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     15058
           1       1.00      1.00      1.00     13676

    accuracy                           1.00     28734
   macro avg       1.00      1.00      1.00     28734
weighted avg       1.00      1.00      1.00     28734

Confusion Matrix:
 [[15058     0]
 [    0 13676]]


Evaluation for testing data

In [47]:
print("Evaluation for Logistic Regression")
y_pred=log_model.predict(x_val)
print(f"Accuracy is ",accuracy_score(y_val,y_pred))
print("Classification Report:\n", classification_report(y_val, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_val, y_pred))

Evaluation for Logistic Regression
Accuracy is  1.0
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      3773
           1       1.00      1.00      1.00      3411

    accuracy                           1.00      7184
   macro avg       1.00      1.00      1.00      7184
weighted avg       1.00      1.00      1.00      7184

Confusion Matrix:
 [[3773    0]
 [   0 3411]]



Making a Predictive System

In [60]:
X_new = x_val[3]

prediction = log_model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake
